In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.layers import Conv2D
from time import time
from datetime import datetime
from keras.callbacks import TensorBoard, ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
df1 = pd.read_hdf('df_dataDOL1.hdf', key='hdf')
df2 = pd.read_hdf('df_dataDOL2.hdf', key='hdf')
df3 = pd.read_hdf('df_dataDOL3.hdf', key='hdf')

df_total = pd.concat([df1,df2,df3],  ignore_index=True)
df_total.head()

,Data,Contratos Abertos,Contratos Fechados,Número Negócios,Contratos Negociados,Volume,Abertura,Mínimo,Máximo,Médio,Último Preço,Ajuste,Var pontos,Última oferta de compra,Última oferta de venda
0,29/07/2019,851537,719017,27192,226625,42939771125,3.7850,3.7775,3.8015,3.789499,3.7820,3.784797,145520,3.7820,3.7825
1,26/07/2019,875047,851537,32128,309480,58414828000,3.7800,3.7610,3.7955,3.775030,3.7760,3.770245,125890,3.7765,3.7780
2,25/07/2019,890941,875047,41651,381060,72105906125,3.7680,3.7570,3.8070,3.784490,3.7795,3.782834,129780,3.7795,3.7805
3,24/07/2019,883412,890941,26137,269620,50725252625,3.7675,3.7540,3.7795,3.762721,3.7735,3.769856,31160,3.7735,3.7755
4,23/07/2019,880504,883412,33586,313840,59121712875,3.7450,3.7430,3.7820,3.767634,3.7750,3.772972,356000,3.7750,3.7770


In [3]:
df = df_total.copy()

df['Dif_contratos'] = df['Contratos Abertos'] -df['Contratos Fechados']
df['Dif_minmax'] = (df['Máximo'] - df['Mínimo'])*1000
df['Dif_abert_ultimo'] = (df['Último Preço'] - df['Abertura'])*1000
df['Dif_abert_ajuste'] = (df['Ajuste'] - df['Abertura'])*1000
df['Dif_abert_min'] = (df['Abertura'] - df['Mínimo'])*1000
df['Dif_abert_max'] = (df['Abertura'] - df['Máximo'])*1000
df['Dif_abert_medio'] = (df['Abertura'] - df['Médio'])*1000
df['Dif_ajuste_ultimo'] = (df['Ajuste'] - df['Último Preço'])*1000
df['Dif_ajuste_medio'] = (df['Ajuste'] - df['Médio'])*1000
df['Dif_ultimacomp_ultimavend'] = (df['Última oferta de venda'] - df['Última oferta de compra'])*1000

df['Contratos Negociados'] = df['Contratos Negociados']/10000
df['Número Negócios'] = df['Número Negócios']/100
df['Volume'] = df['Volume']/1000000000


df = df.drop(columns = ['Contratos Abertos', 'Contratos Fechados'])


df_feature = df.set_index('Data') #Usado depois do treinamento


df.head()

,Data,Número Negócios,Contratos Negociados,Volume,Abertura,Mínimo,Máximo,Médio,Último Preço,Ajuste,...,Dif_contratos,Dif_minmax,Dif_abert_ultimo,Dif_abert_ajuste,Dif_abert_min,Dif_abert_max,Dif_abert_medio,Dif_ajuste_ultimo,Dif_ajuste_medio,Dif_ultimacomp_ultimavend
0,29/07/2019,271.92,22.6625,42.939771,3.7850,3.7775,3.8015,3.789499,3.7820,3.784797,...,132520,24.0,-3.0,-0.203,7.5,-16.5,-4.499,2.797,-4.702,0.5
1,26/07/2019,321.28,30.9480,58.414828,3.7800,3.7610,3.7955,3.775030,3.7760,3.770245,...,23510,34.5,-4.0,-9.755,19.0,-15.5,4.970,-5.755,-4.785,1.5
2,25/07/2019,416.51,38.1060,72.105906,3.7680,3.7570,3.8070,3.784490,3.7795,3.782834,...,15894,50.0,11.5,14.834,11.0,-39.0,-16.490,3.334,-1.656,1.0
3,24/07/2019,261.37,26.9620,50.725253,3.7675,3.7540,3.7795,3.762721,3.7735,3.769856,...,-7529,25.5,6.0,2.356,13.5,-12.0,4.779,-3.644,7.135,2.0
4,23/07/2019,335.86,31.3840,59.121713,3.7450,3.7430,3.7820,3.767634,3.7750,3.772972,...,-2908,39.0,30.0,27.972,2.0,-37.0,-22.634,-2.028,5.338,2.0


In [4]:
df_abert_old = df.copy()

#1 se o Ajuste for maior que a abertura


df_abert_old['target'] = df_abert_old.Ajuste.shift(-1) - df_abert_old['Ajuste']
df_abert_old['target'] = df_abert_old['target'].apply(lambda x: 1 if x > 0 else 0)
#df_abert_old = df_abert_old.drop(df_abert_old.tail(1).index,inplace=True)

df_abert_old['Média'] = (df_abert_old['Último Preço'] + df_abert_old['Abertura'] + df_abert_old['Máximo'] + df_abert_old['Mínimo'] + df_abert_old['Médio'] + df_abert_old['Ajuste'] + df_abert_old['Última oferta de compra']+ df_abert_old['Última oferta de venda'])/8
df_abert_old = df_abert_old.drop(columns = ['Último Preço','Abertura','Máximo','Mínimo','Médio','Última oferta de compra','Última oferta de venda','Contratos Negociados', 'Ajuste'])


df_change = df_abert_old[['target']]
df_abert_old = df_abert_old.drop(columns = 'target')
df_abert_old['target'] = df_change

df_abert_old.head() 

,Data,Número Negócios,Volume,Var pontos,Dif_contratos,Dif_minmax,Dif_abert_ultimo,Dif_abert_ajuste,Dif_abert_min,Dif_abert_max,Dif_abert_medio,Dif_ajuste_ultimo,Dif_ajuste_medio,Dif_ultimacomp_ultimavend,Média,target
0,29/07/2019,271.92,42.939771,145520,132520,24.0,-3.0,-0.203,7.5,-16.5,-4.499,2.797,-4.702,0.5,3.785600,0
1,26/07/2019,321.28,58.414828,125890,23510,34.5,-4.0,-9.755,19.0,-15.5,4.970,-5.755,-4.785,1.5,3.776534,1
2,25/07/2019,416.51,72.105906,129780,15894,50.0,11.5,14.834,11.0,-39.0,-16.490,3.334,-1.656,1.0,3.779853,0
3,24/07/2019,261.37,50.725253,31160,-7529,25.5,6.0,2.356,13.5,-12.0,4.779,-3.644,7.135,2.0,3.769510,1
4,23/07/2019,335.86,59.121713,356000,-2908,39.0,30.0,27.972,2.0,-37.0,-22.634,-2.028,5.338,2.0,3.767201,0


In [5]:
sc = StandardScaler()
df_abert_old.iloc[:,1:-1]  = sc.fit_transform(df_abert_old.iloc[:,1:-1])



df_moeda = pd.read_csv('moedas_features.csv')

sc = StandardScaler()
df_moeda.iloc[:,1:] = sc.fit_transform(df_moeda.iloc[:,1:])

pca = PCA(n_components=3)
dataset = pca.fit_transform(df_moeda.iloc[:,1:])
df_moeda_pca = pd.DataFrame(dataset)
df_moeda_pca['Data'] = df_moeda.iloc[:,0:1]




df_abert = pd.merge(df_moeda_pca, df_abert_old, on = 'Data')



df_abert.head()

,0,1,2,Data,Número Negócios,Volume,Var pontos,Dif_contratos,Dif_minmax,Dif_abert_ultimo,Dif_abert_ajuste,Dif_abert_min,Dif_abert_max,Dif_abert_medio,Dif_ajuste_ultimo,Dif_ajuste_medio,Dif_ultimacomp_ultimavend,Média,target
0,1.498562,2.807394,0.723413,14/06/2019,2.866027,2.605075,1.099924,-0.181265,1.312951,1.557156,-0.120061,-0.332606,-2.054526,-1.942578,-0.196842,-0.178956,-0.012159,2.178212,0
1,2.236543,1.093141,0.813228,13/06/2019,2.062243,2.314778,-0.649583,-0.483705,-0.004695,-0.058399,-0.185437,0.076400,0.075670,0.280736,-0.182731,-0.177075,-0.011627,2.136875,0
2,1.688401,2.248905,0.239471,12/06/2019,2.508010,2.488214,-0.637310,-0.152949,0.440706,0.335639,-0.207215,0.785345,0.125209,0.694360,-0.223930,-0.186347,-0.012159,2.146036,0
3,2.172415,1.142010,0.374627,11/06/2019,1.687984,1.587085,0.608670,-0.243370,0.459265,-0.964686,-0.258551,1.303420,0.571064,1.353530,-0.211288,-0.217766,-0.011627,2.150590,1
4,2.012423,1.412341,1.183180,10/06/2019,1.377429,1.242700,-0.457538,-0.009080,0.069538,-0.334226,-0.229939,0.676277,0.521525,0.804702,-0.213692,-0.205747,-0.010033,2.179808,0


In [6]:
df_abert['Data'] = df_abert['Data'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
df_abert = df_abert.set_index('Data')
df_abert = df_abert.sort_index()
index = df_abert.index

In [28]:
y = df_abert[['target']]
X = df_abert.drop(columns = ['target'])
X_columns = df_abert.drop(columns = ['target'])

sc = StandardScaler()
X = sc.fit_transform(X)
#X = X.values
X

df_final = pd.DataFrame(X,index = index)

front = 3500

df_final = df_final.iloc[front:]
y = y.iloc[front:]

Teste para descobrir qual fatia dos dados tem melhor resultado com Random Forest

dict_res = {}

for i in range(0,4000,100):

    y = df_abert[['target']]
    X = df_abert.drop(columns = ['target'])
    X_columns = df_abert.drop(columns = ['target'])

    sc = StandardScaler()
    X = sc.fit_transform(X)
    #X = X.values
    X

    df_final = pd.DataFrame(X,index = index)

    df_final = df_final.iloc[i:]
    y = y.iloc[i:]
    
    res_total = 0
    soma = 0
    for k in range(1,10):
        x_train, x_test, y_train, y_test = train_test_split(df_final, y, test_size=0.2, shuffle = False)

        r = RandomForestClassifier()
        r.fit(x_train, y_train)  

        predictions = r.predict(x_test)

        res = accuracy_score(y_test, predictions)
        res_total +=res
        soma+=1
        
    dict_res[i] = res_total/soma

---

Verificação da previsão com quantos pontos teria ganhado de fato

1) Usando apenas a flag

2) Usando as maiores probabilidades

In [29]:
x_train, x_test, y_train, y_test = train_test_split(df_final, y, test_size=0.2, shuffle = False)

In [30]:
r = RandomForestClassifier()
r.fit(x_train, y_train)  

predictions = r.predict(x_test)

accuracy_score(y_test, predictions)

c:\users\choice\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\choice\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.823076923076923

---

Predição utilizando o dataframe inicial para ver quanto teria ganho de fato

In [101]:
index_target = y_test.index
df_target = pd.DataFrame(predictions, index = index_target)
df_target.columns = ['target']

In [102]:
df_inicial_target = pd.merge(df_feature,df_target, left_index = True, right_index = True)

In [103]:
#Se o target é 1 significa que o ajuste do dia seguinte será menor que o do dia anterior

df = df_inicial_target.copy()
df.head()

,Número Negócios,Contratos Negociados,Volume,Abertura,Mínimo,Máximo,Médio,Último Preço,Ajuste,Var pontos,...,Dif_minmax,Dif_abert_ultimo,Dif_abert_ajuste,Dif_abert_min,Dif_abert_max,Dif_abert_medio,Dif_ajuste_ultimo,Dif_ajuste_medio,Dif_ultimacomp_ultimavend,target
Data,,,,,,,,,,,,,,,,,,,,,
2018-09-26,470.11,45.6230,92.349003,4.0795,4.0095,4.0940,4.048352,4.0350,4.031807,510890,...,84.5,-44.5,-47.693,70.0,-14.5,31.148,-3.193,-16.545,1.0,1
2018-09-27,467.79,58.5090,117.146814,4.0400,3.9665,4.0550,4.004403,4.0145,3.993942,378650,...,88.5,-25.5,-46.058,73.5,-15.0,35.597,-20.558,-10.461,0.5,0
2018-09-28,30.94,2.1310,4.283738,4.0370,3.9995,4.0505,4.020401,4.0035,4.003900,99580,...,51.0,-33.5,-33.100,37.5,-13.5,16.599,0.400,-16.501,0.5,1
2018-10-01,327.94,32.9775,53.623388,3.2630,3.2360,3.2655,3.252120,3.2375,3.242120,175170,...,29.5,-25.5,-20.880,27.0,-2.5,10.880,4.620,-10.000,0.0,0
2018-10-04,487.71,46.8995,80.253239,3.4155,3.4075,3.4420,3.422349,3.4150,3.412088,102320,...,34.5,-0.5,-3.412,8.0,-26.5,-6.849,-2.912,-10.261,0.0,1


In [ ]:
def ajuste_valor_real(num):
    

In [104]:
df.loc[(df.target == 1), 'teste'] = df.Ajuste.shift(-1) - df.Abertura.shift(-1)
df.loc[(df.target == 0), 'teste'] = df.Abertura.shift(-1) - df.Ajuste.shift(-1)

In [105]:
#df.loc[(df.target == 1) & (df.Abertura.shift(-1) < df.Ajuste), 'teste'] = df.Ajuste.shift(-1) - df.Abertura.shift(-1)
df_teste = df
df_teste = df_teste.fillna(0)

In [108]:
df_teste['teste'].sum()

3.728140000000005

In [109]:
df_teste

,Número Negócios,Contratos Negociados,Volume,Abertura,Mínimo,Máximo,Médio,Último Preço,Ajuste,Var pontos,...,Dif_abert_ultimo,Dif_abert_ajuste,Dif_abert_min,Dif_abert_max,Dif_abert_medio,Dif_ajuste_ultimo,Dif_ajuste_medio,Dif_ultimacomp_ultimavend,target,teste
Data,,,,,,,,,,,,,,,,,,,,,
2018-09-26,470.11,45.6230,92.349003,4.0795,4.0095,4.0940,4.048352,4.0350,4.031807,510890,...,-44.5,-47.693,70.0,-14.5,31.148,-3.193,-16.545,1.0,1,-0.046058
2018-09-27,467.79,58.5090,117.146814,4.0400,3.9665,4.0550,4.004403,4.0145,3.993942,378650,...,-25.5,-46.058,73.5,-15.0,35.597,-20.558,-10.461,0.5,0,0.033100
2018-09-28,30.94,2.1310,4.283738,4.0370,3.9995,4.0505,4.020401,4.0035,4.003900,99580,...,-33.5,-33.100,37.5,-13.5,16.599,0.400,-16.501,0.5,1,-0.020880
2018-10-01,327.94,32.9775,53.623388,3.2630,3.2360,3.2655,3.252120,3.2375,3.242120,175170,...,-25.5,-20.880,27.0,-2.5,10.880,4.620,-10.000,0.0,0,0.003412
2018-10-04,487.71,46.8995,80.253239,3.4155,3.4075,3.4420,3.422349,3.4150,3.412088,102320,...,-0.5,-3.412,8.0,-26.5,-6.849,-2.912,-10.261,0.0,1,-0.034106
2018-10-05,398.16,38.1885,68.044514,3.5870,3.5500,3.5910,3.563613,3.5580,3.552894,538310,...,-29.0,-34.106,37.0,-4.0,23.387,-5.106,-10.719,0.0,0,-0.034959
2018-10-08,469.76,41.1900,79.558890,3.8445,3.8340,3.8830,3.863019,3.8680,3.879459,683100,...,23.5,34.959,10.5,-38.5,-18.519,11.459,16.440,2.0,0,-0.021145
2018-10-09,387.27,31.7205,65.095391,4.0730,4.0615,4.1355,4.104310,4.0945,4.094145,338670,...,21.5,21.145,11.5,-62.5,-31.310,-0.355,-10.165,0.5,1,0.018575
2018-10-10,443.13,40.5485,76.150600,3.7420,3.7300,3.7750,3.756025,3.7620,3.760575,502320,...,20.0,18.575,12.0,-33.0,-14.025,-1.425,4.550,0.5,0,0.026687
